In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
key = os.getenv("OPENAI_API_KEY")
api_base_url = os.getenv("OPENAI_API_BASE")



In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [13]:


llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo-16k",
    temperature=0.7
)

response = llm([HumanMessage(content="Hello! What can you do?")])
print(response.content)

Hello! I am a language model AI trained to assist with various tasks such as answering questions, providing information, and engaging in conversation. How can I help you today?


In [21]:
from langchain.llms import openai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [22]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [23]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [24]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [25]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [26]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [31]:
chain1 = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [32]:
chain2 = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [38]:
quiz_evaluation_chain = SequentialChain(chains=[chain1, chain2], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [34]:
file_path = "/home/owais/MyProjects/GEN_AI/data.txt"

In [35]:
with open(file_path, 'r') as file:
    text = file.read()

In [36]:
text

'The first example of an algorithmically generated media is likely the Markov chain. Markov chains have long been used to model natural languages since their development by Russian mathematician Andrey Markov in the early 20th century. Markov published his first paper on the topic in 1906,[27][28] and analyzed the pattern of vowels and consonants in the novel Eugeny Onegin using Markov chains. Once a Markov chain is trained on a text corpus, it can then be used as a probabilistic text generator.[29][30]\n\nComputers were needed to go beyond Markov chains. By the early 1970s, Harold Cohen was creating and exhibiting generative AI works created by AARON, the computer program Cohen created to generate paintings.[31]\n\nThe terms generative AI planning or generative planning were used in the 1980s and 1990s to refer to AI planning systems, especially computer-aided process planning, used to generate sequences of actions to reach a specified goal.[32][33] Generative AI planning systems used

In [37]:
NUMBER = 10
SUBJECT = "Generative AI"
TONE = "normal"

In [39]:
with get_openai_callback() as cb:
    response=quiz_evaluation_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/tmp/ipykernel_21908/550409977.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=quiz_evaluation_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The first example of an algorithmically generated media is likely the Markov chain. Markov chains have long been used to model natural languages since their development by Russian mathematician Andrey Markov in the early 20th century. Markov published his first paper on the topic in 1906,[27][28] and analyzed the pattern of vowels and consonants in the novel Eugeny Onegin using Markov chains. Once a Markov chain is trained on a text corpus, it can then be used as a probabilistic text generator.[29][30]

Computers were needed to go beyond Markov chains. By the early 1970s, Harold Cohen was creating and exhibiting generative AI works created by AARON, the computer program Cohen created to generate paintings.[31]

The terms generative AI planning or generative planning were used in the 1980s and 1990s to refer to AI planning systems, especially computer-aided process planning, used 

In [40]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2656
Prompt Tokens:1777
Completion Tokens:879
Total Cost:0.0


In [41]:
response

{'text': 'The first example of an algorithmically generated media is likely the Markov chain. Markov chains have long been used to model natural languages since their development by Russian mathematician Andrey Markov in the early 20th century. Markov published his first paper on the topic in 1906,[27][28] and analyzed the pattern of vowels and consonants in the novel Eugeny Onegin using Markov chains. Once a Markov chain is trained on a text corpus, it can then be used as a probabilistic text generator.[29][30]\n\nComputers were needed to go beyond Markov chains. By the early 1970s, Harold Cohen was creating and exhibiting generative AI works created by AARON, the computer program Cohen created to generate paintings.[31]\n\nThe terms generative AI planning or generative planning were used in the 1980s and 1990s to refer to AI planning systems, especially computer-aided process planning, used to generate sequences of actions to reach a specified goal.[32][33] Generative AI planning sys

In [47]:
quiz = json.loads(response.get("quiz"))

In [48]:
quiz

{'1': {'mcq': 'Which Russian mathematician is credited with the development of Markov chains?',
  'options': {'a': 'Andrey Markov',
   'b': 'Harold Cohen',
   'c': 'Eugeny Onegin',
   'd': 'Variational autoencoder'},
  'correct': 'a'},
 '2': {'mcq': 'What was the first text corpus that Andrey Markov analyzed using Markov chains?',
  'options': {'a': 'Eugeny Onegin',
   'b': 'AARON',
   'c': 'GPT-2',
   'd': 'Transformer network'},
  'correct': 'a'},
 '3': {'mcq': 'Which computer program was created by Harold Cohen to generate paintings?',
  'options': {'a': 'AARON',
   'b': 'Generative adversarial network',
   'c': 'Deep learning',
   'd': 'State space search'},
  'correct': 'a'},
 '4': {'mcq': 'What technology was considered relatively mature in the early 1990s for generating crisis action plans for military use?',
  'options': {'a': 'Variational autoencoder',
   'b': 'Generative AI planning systems',
   'c': 'Markov chains',
   'd': 'GPT-1'},
  'correct': 'b'},
 '5': {'mcq': 'Which t

In [51]:
review = response.get("review")

In [52]:
review

'The quiz is highly complex, focusing on advanced topics in generative AI. The questions require deep understanding of concepts and terminology. To better fit student abilities, simplify questions, provide more context, and use simpler language.'

In [53]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [54]:
quiz_table_data

[{'MCQ': 'Which Russian mathematician is credited with the development of Markov chains?',
  'Choices': 'a: Andrey Markov | b: Harold Cohen | c: Eugeny Onegin | d: Variational autoencoder',
  'Correct': 'a'},
 {'MCQ': 'What was the first text corpus that Andrey Markov analyzed using Markov chains?',
  'Choices': 'a: Eugeny Onegin | b: AARON | c: GPT-2 | d: Transformer network',
  'Correct': 'a'},
 {'MCQ': 'Which computer program was created by Harold Cohen to generate paintings?',
  'Choices': 'a: AARON | b: Generative adversarial network | c: Deep learning | d: State space search',
  'Correct': 'a'},
 {'MCQ': 'What technology was considered relatively mature in the early 1990s for generating crisis action plans for military use?',
  'Choices': 'a: Variational autoencoder | b: Generative AI planning systems | c: Markov chains | d: GPT-1',
  'Correct': 'b'},
 {'MCQ': 'Which technology was used to generate process plans for manufacturing in the 1980s and 1990s?',
  'Choices': 'a: Constra

In [55]:
quiz = pd.DataFrame(quiz_table_data)

In [56]:
quiz.head()

,MCQ,Choices,Correct
0,Which Russian mathematician is credited with t...,a: Andrey Markov | b: Harold Cohen | c: Eugeny...,a
1,What was the first text corpus that Andrey Mar...,a: Eugeny Onegin | b: AARON | c: GPT-2 | d: Tr...,a
2,Which computer program was created by Harold C...,a: AARON | b: Generative adversarial network |...,a
3,What technology was considered relatively matu...,a: Variational autoencoder | b: Generative AI ...,b
4,Which technology was used to generate process ...,a: Constraint satisfaction | b: Neural network...,a


In [57]:
quiz.to_csv(f"{SUBJECT}.csv", index=False)